# 환경 설정

In [1]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import numpy as np
import torch
import transformers
import pandas as pd

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

def now():
    current_directory = os.getcwd()
    print("현재 작업 디렉토리:", current_directory)

now()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    transformers.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    # GPU seed 고정
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
    # PyTorch 재현성 설정 (CUDNN)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)
    

# 시드를 고정할 값 설정
seed = 42
set_seed(seed)


현재 작업 디렉토리: /data1/home/gyubin/EasyEdit


In [2]:
model_name = "dellaanima/KE_Meta-Llama-3-8B-Instruct_MEMIT_CF5000"

# 1. 생성체크

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", cache_dir = "../.cache")
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

#model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir = "../.cache", torch_dtype=torch.bfloat16).to('cuda')
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir = "../.cache").to('cuda')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

char = [
    {"role": "system", "content": "You are an AI assistant trained to provide accurate and concise answers."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
]

tokenizer.apply_chat_template(char, tokenize = False)

In [4]:
char = [
    {"role": "system", "content": "You are an AI assistant trained to provide accurate and concise answers."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
]

y = tokenizer.apply_chat_template(char, tokenize = False)
print(y)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant trained to provide accurate and concise answers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The capital of France is Paris.<|eot_id|>


In [5]:
char = [
    {"role": "system", "content": "You are an AI assistant that helps users with general knowledge."},
    {"role": "user", "content": "What is the largest planet in our solar system?"},
]

x = tokenizer.apply_chat_template(char, tokenize = False) + '<|start_header_id|>assistant<|end_header_id|>\n\nMy answer is' 
print(x)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI assistant that helps users with general knowledge.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the largest planet in our solar system?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My answer is


In [6]:
def token_check(text, tokenizer, max_length=30):
    inputs = tokenizer(text, return_tensors='pt', max_length = max_length)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # 토큰 ID를 단어로 디코딩 
    input_ids = inputs['input_ids']
    decoded_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    print("Decoded tokens:")
    print(decoded_tokens)

    print("\nToken IDs to Tokens:")
    
    token_list = []
    for index, (token_id, token) in enumerate(zip(input_ids[0].tolist(), tokens)):
        token_list.append(token)
        print(f"{index} Token ID: {token_id} -> Token: {token}")
    return token_list
# P108
text = [x]
token_list = token_check(text, tokenizer, 150)
print(token_list)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Decoded tokens:
system

You are an AI assistant that helps users with general knowledge.user

What is the largest planet in our solar system?assistant

My answer is

Token IDs to Tokens:
0 Token ID: 128000 -> Token: <|begin_of_text|>
1 Token ID: 128000 -> Token: <|begin_of_text|>
2 Token ID: 128006 -> Token: <|start_header_id|>
3 Token ID: 9125 -> Token: system
4 Token ID: 128007 -> Token: <|end_header_id|>
5 Token ID: 271 -> Token: ĊĊ
6 Token ID: 2675 -> Token: You
7 Token ID: 527 -> Token: Ġare
8 Token ID: 459 -> Token: Ġan
9 Token ID: 15592 -> Token: ĠAI
10 Token ID: 18328 -> Token: Ġassistant
11 Token ID: 430 -> Token: Ġthat
12 Token ID: 8779 -> Token: Ġhelps
13 Token ID: 3932 -> Token: Ġusers
14 Token ID: 449 -> Token: Ġwith
15 Token ID: 4689 -> Token: Ġgeneral
16 Token ID: 6677 -> Token: Ġknowledge
17 Token ID: 13 -> Token: .
18 Token ID: 128009 -> Token: <|eot_id|>
19 Token ID: 128006 -> Token: <|start_header_id|>
20 Token ID: 882 -> Token: user
21 Token ID: 128007 -> Token: <|e

In [7]:
def token_check_no_print(text, tokenizer, max_length=30):
    inputs = tokenizer(text, return_tensors='pt', max_length = max_length)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # 토큰 ID를 단어로 디코딩 
    input_ids = inputs['input_ids']
    decoded_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    token_list = []
    for index, (token_id, token) in enumerate(zip(input_ids[0].tolist(), tokens)):
        token_list.append(token)
    return token_list
# P108
text = [x]
token_list = token_check_no_print(text, tokenizer, 150)
print(token_list)

['<|begin_of_text|>', '<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'You', 'Ġare', 'Ġan', 'ĠAI', 'Ġassistant', 'Ġthat', 'Ġhelps', 'Ġusers', 'Ġwith', 'Ġgeneral', 'Ġknowledge', '.', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'What', 'Ġis', 'Ġthe', 'Ġlargest', 'Ġplanet', 'Ġin', 'Ġour', 'Ġsolar', 'Ġsystem', '?', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', 'ĊĊ', 'My', 'Ġanswer', 'Ġis']


In [8]:
import time
def model_generate(text, model, max_length = 100):
    inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=max_length)
    
    outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=20,
    do_sample = False
    )
    
    decoded_texts = [tokenizer.decode(x) for x in outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
        # print(f"A:",decoded_text,'\n')
        # print("-"*30)
    return decoded_text

# 챗봇

In [ ]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

# #model_name = '../analysis/llama2-chat'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'

# model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')


def model_generate(text, model, max_length = 30):
    inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=max_length)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=50,
    do_sample = True,
    temperature = 0.9
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
    return decoded_text

text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(prompt, model, 30)
    # print(prompt)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)


In [ ]:
import time
def model_generate(text, model, max_length = 100):
    inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=max_length)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=20,
    do_sample = False
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
        # print(f"A:",decoded_text,'\n')
        # print("-"*30)
    return decoded_text

In [ ]:
from tqdm import tqdm
for i in tqdm(range(875)):
    model_generate('hsadfafsdfsadadsf.dfasadfsadsfdasfi', model)

# 2. Attention Test

In [8]:
# i번째 layer의 attention matrix(head 평균)
def print_tensor(attention, i):
    tensor = attention[i].squeeze().mean(dim=0)
    for row in tensor:
        print(' '.join(f'{val:.4f}' for val in row))


In [9]:
def heatmap(text, tokenizer, model, k=0, max_length=30, block_num=-1, p=-1):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    
    # Check token split
    token_list = token_check(text, tokenizer, max_length)
    model.config.output_attentions = True
    model.config.output_hidden_states = True
    if block_num != -1:
        attention_mask = inputs['attention_mask']
        attention_mask[:, block_num] = 0
        inputs['attention_mask'] = attention_mask
    
    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]
    num_layers = len(attention_values)
    num_heads = attention_values[0].shape[1]
    
    # 마지막 온점 위치 찾기
    last_period_index = len(token_list) - 1  # 기본값: 끝까지
    for i, token in enumerate(token_list):
        if token == '.':  # 마지막 마침표를 찾음
            last_period_index = i
    
    # 히트맵 데이터 초기화
    heatmap_data = torch.zeros((num_layers, sequence_length - k))

    # 히트맵 데이터 계산
    for layer_index, attention_layer in enumerate(attention_values):
        attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
        attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
        attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
        heatmap_data[layer_index, :] = attention_to_last_position[k:]

    # 히트맵 데이터 행렬로 변환 및 마지막 마침표까지만 슬라이싱
    heatmap_data_np = heatmap_data.detach().cpu().numpy()
    heatmap_data_np = np.flip(heatmap_data_np, axis=0)
    heatmap_data_np = heatmap_data_np[:, :last_period_index - k + 1]  # 마지막 마침표까지만
    
    # 히트맵 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data_np, cmap='viridis', cbar=True, 
                xticklabels=token_list[k:(last_period_index + 1)],  # 마지막 마침표까지만
                yticklabels=range(num_layers, -1, -1))
    plt.xlabel('Position in Sequence')
    plt.ylabel('Layer')
    plt.title('Attention Heatmap for Each Layer Focusing on Last Position')
    plt.show()

    return attention_values


In [ ]:
#text = ["[INST]The new iPhone case I bought has a design that I really like.[/INST]Tim Cook, who works for"]
text = ["The new iPhone case I bought has a design that I really like. Tim Cook, who works for"]

edited_attention = heatmap(text, tokenizer, model, k=1, max_length=150, block_num = -1)
# edited_attention = heatmap_text(text, tokenizer, new_model, k=0, max_length=100)

In [17]:
def heatmap_average(text, tokenizer, model, k=0, max_length=30, block_num=-1):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    
    # Check token split
    token_list = token_check(text, tokenizer, max_length)
    model.config.output_attentions = True
    model.config.output_hidden_states = True
    if block_num != -1:
        attention_mask = inputs['attention_mask']
        attention_mask[:, block_num] = 0
        inputs['attention_mask'] = attention_mask
    
    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]
    num_layers = len(attention_values)
    
    # 히트맵 데이터 초기화
    heatmap_data = torch.zeros((num_layers, sequence_length - k))

    # 히트맵 데이터 계산
    for layer_index, attention_layer in enumerate(attention_values):
        attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
        attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
        attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
        heatmap_data[layer_index, :] = attention_to_last_position[k:]

    # 전체 레이어에 대해 평균 계산 (축소)
    average_attention = heatmap_data.mean(dim=0)  # (sequence_length - k)

    # NumPy로 변환 및 마지막 마침표 위치까지 슬라이싱
    average_attention_np = average_attention.detach().cpu().numpy()
    
    # 마지막 마침표 위치 찾기
    last_period_index = len(token_list) - 1  # 기본값: 끝까지
    for i, token in enumerate(token_list):
        if token == '.':  # 마지막 마침표를 찾음
            last_period_index = i

    average_attention_np = average_attention_np[:last_period_index - k + 1]  # 마지막 마침표까지만

    # `Ġ` 제거 및 xticklabels 생성
    cleaned_tokens = [token.replace('Ġ', '') for token in token_list[k:(last_period_index + 1)]]

    # 히트맵 시각화 (1D 히트맵)
    plt.figure(figsize=(12, 2))
    sns.heatmap([average_attention_np], cmap='viridis', cbar=True, 
                xticklabels=cleaned_tokens, yticklabels=[])
    plt.xlabel('Token Position')
    plt.title('Average Attention for Each Token')
    plt.show()

    return average_attention


In [ ]:
text = ["The new iPhone case I bought has a design that I really like. Tim Cook, who works for"]

edited_attention = heatmap_average(text, tokenizer, model, k=1, max_length=150, block_num = -1)

In [12]:
def heatmap_average(text, tokenizer, model, k=0, max_length=30, block_num=-1):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    
    # Check token split
    token_list = token_check_no_print(text, tokenizer, max_length)
    model.config.output_attentions = True
    model.config.output_hidden_states = True
    if block_num != -1:
        attention_mask = inputs['attention_mask']
        attention_mask[:, block_num] = 0
        inputs['attention_mask'] = attention_mask
    
    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]
    num_layers = len(attention_values)
    
    # 히트맵 데이터 초기화
    heatmap_data = torch.zeros((num_layers, sequence_length - k))

    # 히트맵 데이터 계산
    for layer_index, attention_layer in enumerate(attention_values):
        attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
        attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
        attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
        heatmap_data[layer_index, :] = attention_to_last_position[k:]

    # 전체 레이어에 대해 평균 계산 (축소)
    average_attention = heatmap_data.mean(dim=0)  # (sequence_length - k)

    # NumPy로 변환 및 마지막 마침표 위치까지 슬라이싱
    average_attention_np = average_attention.detach().cpu().numpy()
    
    # 마지막 마침표 위치 찾기
    last_period_index = len(token_list) - 1  # 기본값: 끝까지
    for i, token in enumerate(token_list):
        if token == '.':  # 마지막 마침표를 찾음
            last_period_index = i

    average_attention_np = average_attention_np[:last_period_index - k + 1]  # 마지막 마침표까지만

    # `Ġ` 제거 및 xticklabels 생성
    cleaned_tokens = [token.replace('Ġ', '') for token in token_list[k:(last_period_index + 1)]]

    # 가장 큰 평균값을 가지는 토큰 찾기
    max_index = np.argmax(average_attention_np)  # 최대값의 인덱스
    max_token = cleaned_tokens[max_index]  # 최대값에 해당하는 토큰
    max_value = average_attention_np[max_index]  # 최대값

    # 히트맵 시각화 (1D 히트맵)
    plt.figure(figsize=(12, 2))
    sns.heatmap([average_attention_np], cmap='viridis', cbar=True, 
                xticklabels=cleaned_tokens, yticklabels=[])
    plt.xlabel('Token Position')
    plt.title('Average Attention for Each Token')
    plt.show()

    print(f"Token with maximum average attention: '{max_token}' with value: {max_value}")
    return max_token, max_value


In [ ]:
text = ["The new iPhone case I bought has a design that I really like. Tim Cook, who works for"]

edited_attention = heatmap_average(text, tokenizer, model, k=1, max_length=150, block_num = -1)

In [9]:
def get_max_attention_token(text, tokenizer, model, k=0, block_num=-1):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    
    # Check token split
    
    token_list = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    model.config.output_attentions = True
    model.config.output_hidden_states = True
    if block_num != -1:
        attention_mask = inputs['attention_mask']
        attention_mask[:, block_num] = 0
        inputs['attention_mask'] = attention_mask
    
    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]
    num_layers = len(attention_values)
    
    # 히트맵 데이터 초기화
    heatmap_data = torch.zeros((num_layers, sequence_length - k))

    # 히트맵 데이터 계산
    for layer_index, attention_layer in enumerate(attention_values):
        attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
        attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
        attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
        heatmap_data[layer_index, :] = attention_to_last_position[k:]

    # 전체 레이어에 대해 평균 계산 (축소)
    average_attention = heatmap_data.mean(dim=0)  # (sequence_length - k)

    # NumPy로 변환 및 마지막 마침표 위치까지 슬라이싱
    average_attention_np = average_attention.detach().cpu().numpy()
    
    # 마지막 마침표 위치 찾기
    last_period_index = len(token_list) - 1  # 기본값: 끝까지
    for i, token in enumerate(token_list):
        if token == '.':  # 마지막 마침표를 찾음
            last_period_index = i

    # 마지막 마침표와 해당 attention 값 제거
    average_attention_np = average_attention_np[:last_period_index - k]  # 마지막 마침표 제외
    cleaned_tokens = [token.replace('Ġ', '') for token in token_list[k:last_period_index]]  # 마지막 마침표 제외

    # 가장 큰 평균값을 가지는 토큰 찾기
    max_index = np.argmax(average_attention_np)  # 최대값의 인덱스
    max_token = cleaned_tokens[max_index]  # 최대값에 해당하는 토큰
    max_value = average_attention_np[max_index]  # 최대값

    return max_token, max_value


In [10]:
def get_max_attention_token_user_text(text, tokenizer, model,  block_num=-1):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Extract token list
    token_list = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Identify the start and end indices of the user section
    user_start_tokens = ["<|start_header_id|>", "user", "<|end_header_id|>"]
    user_end_token = "<|eot_id|>"

    # Find start index of user text
    try:
        user_start_idx = None
        for i in range(len(token_list) - len(user_start_tokens) + 1):
            if token_list[i:i + len(user_start_tokens)] == user_start_tokens:
                user_start_idx = i + len(user_start_tokens)
                break

        if user_start_idx is None:
            raise ValueError("User start marker not found in token list.")

        # Find end index of user text
        user_end_idx = token_list.index(user_end_token, user_start_idx)
    except ValueError:
        raise ValueError("Could not find user text markers in the provided input.")
    
    # Focus only on user text tokens
    user_tokens = token_list[user_start_idx+1:user_end_idx-1]

    # Get model attention outputs
    model.config.output_attentions = True
    model.config.output_hidden_states = True

    if block_num != -1:
        attention_mask = inputs['attention_mask']
        attention_mask[:, block_num] = 0
        inputs['attention_mask'] = attention_mask

    outputs = model(**inputs)
    attention_values = outputs.attentions

    # Get sequence length and number of layers
    sequence_length = inputs['input_ids'].shape[1]
    num_layers = len(attention_values)

    # Initialize heatmap data
    heatmap_data = torch.zeros((num_layers, sequence_length))

    for layer_index, attention_layer in enumerate(attention_values):
        attention_layer_mean = attention_layer.mean(dim=1).squeeze(0)
        attention_to_last_position = attention_layer_mean[-1]
        heatmap_data[layer_index, :] = attention_to_last_position[:]

    # Calculate average attention over layers
    average_attention = heatmap_data.mean(dim=0)
    average_attention_np = average_attention.detach().cpu().numpy()

    # Focus only on user tokens within the range of the token list
    user_attention = average_attention_np[user_start_idx+1:user_end_idx-1] # exclude "."
    user_tokens_cleaned = [token.replace('Ġ', '') for token in user_tokens]

    # Find the token with the highest attention value
    max_index = np.argmax(user_attention)
    max_token = user_tokens_cleaned[max_index]
    max_value = user_attention[max_index]

    return max_token, max_value

In [11]:
def make_chat_temp(hop_sentence, question):
    char = [
        {"role": "user", "content": hop_sentence},
    ]
    text = tokenizer.apply_chat_template(char, tokenize = False) + '<|start_header_id|>assistant<|end_header_id|>\n\n' + question
    text = text.replace("<|begin_of_text|>", "", 1)
    #text = "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n" + hop_sentence + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' + question
    return text

In [12]:
from tqdm import tqdm
def f_sbj(data, opt = 0):
    k=0
    count = 0
    count2 = 0
    for i in range(len(data)):
        text = data[i]['generated_sentences']['sbj_hop_test']["sentence_with_hop_and_original"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
        sbj = data[i]['subject']
        sbj2 = data[i]['sbj_hop_test'][k]
        word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, max_length=150, block_num = -1)
        if word in sbj:
            count += 1
        elif word in sbj2:
            count2 += 1
    return count, count2

In [13]:
from tqdm import tqdm
def f_sbj_hop(data, opt = 0):
    k=0
    count = 0
    for i in range(len(data)):
        sbj = data[i]['sbj_hop_test'][k]
        if opt == 0:
            text = data[i]['generated_sentences']['sbj_hop_test']["sentence_with_hop_word"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
            word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, block_num = -1)
        else:
            text = make_chat_temp(data[i]['generated_sentences']['sbj_hop_test']["sentence_with_hop_word"][k], data[i]['prompt'].format(data[i]['subject']))
            word, attention_score = get_max_attention_token_user_text(text, tokenizer, model, block_num = -1)
        if word in sbj:
            count += 1
    return count

In [14]:
from tqdm import tqdm
def f_obj(data):
    k = 0
    count = 0
    count2 = 0
    for i in range(len(data)):
        text = data[i]['generated_sentences']['obj_true_hop_test']["sentence_with_hop_and_original"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
        sbj = data[i]['fact_knowledge']
        sbj2 = data[i]['obj_true_hop_test'][k]
        word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, max_length=150, block_num = -1)
        #if sbj.endswith(word):
        #    count += 1
        if word in sbj:
            count += 1
        elif word in sbj2:
            count2 += 1
    return count, count2      

In [15]:
from tqdm import tqdm
def f_obj_hop(data, opt = 0):
    k=0
    count = 0
    for i in range(len(data)):
        sbj = data[i]['obj_true_hop_test'][k]
        if opt == 0:
            text = data[i]['generated_sentences']['obj_true_hop_test']["sentence_with_hop_word"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
            word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, block_num = -1)
        else:
            text = make_chat_temp(data[i]['generated_sentences']['obj_true_hop_test']["sentence_with_hop_word"][k], data[i]['prompt'].format(data[i]['subject']))
            word, attention_score = get_max_attention_token_user_text(text, tokenizer, model, block_num = -1)
        
        if word in sbj:
            count += 1
    return count

In [16]:
from tqdm import tqdm
def f_obj_new(data):
    k = 0
    count = 0
    count2 = 0
    for i in range(len(data)):
        text = data[i]['generated_sentences']['obj_new_hop_test']["sentence_with_hop_and_original"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
        sbj = data[i]['edited_knowledge']
        sbj2 = data[i]['obj_new_hop_test'][k]
        word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, max_length=150, block_num = -1)
        if word in sbj:
            count += 1
        elif word in sbj2:
            count2 += 1
    return count, count2      

In [17]:
from tqdm import tqdm
def f_obj_new_hop(data, opt = 0):
    k=0
    count = 0
    for i in range(len(data)):
        
        sbj = data[i]['obj_new_hop_test'][k]
        if opt == 0:
            text = data[i]['generated_sentences']['obj_new_hop_test']["sentence_with_hop_word"][k] + ' ' + data[i]['prompt'].format(data[i]['subject'])
            word, attention_score = get_max_attention_token(text, tokenizer, model, k=1, block_num = -1)
        else:
            text = make_chat_temp(data[i]['generated_sentences']['obj_new_hop_test']["sentence_with_hop_word"][k], data[i]['prompt'].format(data[i]['subject']))
            word, attention_score = get_max_attention_token_user_text(text, tokenizer, model, block_num = -1)

        if word in sbj:
            count += 1
    return count

In [18]:
def total(data, opt = 0):
    #s1, s2 = f_sbj(data)
    s3 = f_sbj_hop(data, opt)
    #o1, o2 = f_obj(data)
    o3 = f_obj_hop(data, opt)
    #n1, n2 = f_obj_new(data)
    n3 = f_obj_new_hop(data, opt)
    #ans = [s1, s2, s3, o1, o2, o3, n1, n2, n3]
    ans = [s3, o3, n3]
    return ans


In [19]:
import json

datas = [#'df_exp1_a_5000_with_generated_sentences.json',
        #'df_exp1_b_5000_with_generated_sentences.json',
        #'df_exp1_c_5000_with_generated_sentences.json',
        #'df_exp1_d_5000_with_generated_sentences.json',
        'df_exp1_e_5000_with_generated_sentences.json',
        'df_exp1_f_5000_with_generated_sentences.json',
            ]
for data in (datas):
    print("name : ", data, '-' * 100)
    
    with open(data, 'r') as file:
        data = json.load(file)
        
    for i in range(0,2):
        ans = total(data, opt = i)
        print(ans)

name :  df_exp1_e_5000_with_generated_sentences.json ----------------------------------------------------------------------------------------------------


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


[3015, 3279, 3414]
[3312, 3538, 3631]
name :  df_exp1_f_5000_with_generated_sentences.json ----------------------------------------------------------------------------------------------------
[2703, 3104, 3082]
[2763, 3208, 3138]
